In [14]:
from extract_parquet import read_parquet
import cmbagent 
from make_prompts import make_prompt
from make_prompts import extract_first_function
from make_prompts import extract_python_definition
from IPython.display import Code, display   # built-in in Jupyter
from ins import evaluate

In [2]:
# Example with a single file
df = read_parquet("data/public-00000-of-00001.parquet")


In [3]:
df

,problem_id,domain,difficulty_level,problem,solution,answer,code_answer_requirements,reference_implementation
0,A 3-State QM Problem,QM,2,The Hamiltonian of a three-level system is giv...,The eigenstates are easily found to be $\frac{...,\begin{equation*}\n \boxed{\langle E\rangle...,Provide the answer in the form of \texttt{pyth...,\begin{python}\ndef expectation_value(A: float...
1,Bias of a Sampled Halo Field,Cosmology,5,"In cosmology, large-scale cosmological dark-ma...",\textbf{Detailed Steps:}\nThe solution to this...,The bias of the sampled halo field is given by...,Provide the answer in the form of the \texttt{...,\begin{python}\nfrom scipy.stats import norm\n...
2,Blackbody in d Dimensions,Stat Mech,1,Assume we live in a 4+1 dimensional spacetime....,The density of states scales as $k^{D-1}dk$ in...,$\boxed{n=5}.$,Provide the answer in a form of \texttt{python...,\begin{python}\ndef answer() -> float:\n re...
3,Boosted Parabolic Trajectory,Classical Mechanics,1,Consider a situation where a space-probe very ...,Conservation of energy gives $\frac{1}{2}m(v_e...,\begin{equation*}\n \boxed{v_\infty = \delt...,Provide the answer in the form of \texttt{pyth...,\begin{python}\nfrom math import sqrt\ndef spe...
4,Dark Matter Capture as a Function of Time,Cosmology,2,Suppose $C$ is the capture rate of dark matter...,We can integrate by quadrature.\n\begin{equati...,\begin{equation}\n\boxed{N=\frac{\sqrt{C}}{\sq...,Provide the answer in the form of the \texttt{...,"\begin{python}\nfrom math import sqrt, exp\n\n..."
5,One-Pole Problem,Cosmology,5,Consider the conformally coupled scalar field ...,\begin{figure}\n\begin{centering}\n\begin{tikz...,\begin{equation}\n\boxed{|\beta|\approx\frac{\...,Provide the answer in the form of the \texttt{...,"\begin{python}\nfrom numpy import sqrt, exp, p..."
6,Scalar Particle Scattering,HET,3,Consider\n\begin{equation}\n\mathcal{L} = \lef...,\textbf{Detailed Steps:}\nThe amplitude for th...,\[\n\left( \frac{d\sigma}{d\Omega} \right)_{\t...,Provide the answer in the form of the \texttt{...,"\begin{python}\nfrom math import sqrt, pi\ndef..."
7,SHO Vacuum Entanglement,QM,4,Consider a coupled simple harmonic oscillator ...,Diagonalize the original Hamiltonian \n\begin{...,\begin{equation}\nS = \boxed{-\ln\left(\frac{4...,Provide the answer in the form of the \texttt{...,"\begin{python}\nfrom math import sqrt, log\nde..."
8,Slow-Roll Inflation,Cosmology,3,For the action\n\begin{equation}\nS = \int dt ...,The equation of motion is\n\begin{equation}\n\...,\[\n\phi = \sqrt{2q} M_P \ln \left\{ \exp \lef...,Provide the answer in the form of the \texttt{...,\begin{python}\nimport numpy as np\nfrom numpy...
9,SUSY-Symmetry,HET,4,Consider the theory\n\begin{equation}\n\mathca...,Denoting the variation $\left(\delta_{\eta}\ph...,\begin{equation}\n\boxed{\delta_{\eta}\phi=-\s...,Provide the answer in the form of the \texttt{...,\begin{python}\nfrom math import sqrt\ndef fin...


In [4]:
# --- usage ---------------------------------------------------------------
i =1                                        # whichever row you want
prompt = make_prompt(df.iloc[i])             # combined string for that row
print(prompt)

### Problem
In cosmology, large-scale cosmological dark-matter halo fields are biased tracers of the underlying Gaussian matter density $\delta_m$. Assume we have a sample $\delta_m$. We simulate a halo number density field by taking $n(\mathbf{x}) = \bar{n}\max(0,1+b\delta_m(\mathbf{x}))$, where bare number density $\bar{n}$ and bare bias $b$ are specified constants. What is the bias of the sampled halo field? Derive an equation to evaluate the bias which depends on the bare bias and the variance in each pixel.

### Requirements
Provide the answer in the form of the \texttt{python} code. Implement the following function.
\begin{python}
#let b_in stand for bare bias
def b_eff(sigma: float, b_in:float) -> float:
    pass
\end{python}


In [15]:
reference_code = extract_python_definition(df.reference_implementation.iloc[i])

In [9]:
def get_response(work_dir = "./evals", agent = 'engineer', agent_model = 'gpt-4.1'):

    results = cmbagent.one_shot(prompt,
                       max_rounds=50,
                       agent=agent,
                       engineer_model=agent_model,
                       work_dir=work_dir,
                      )

    for msg in results['chat_history'][::-1]:
        # print("\nin cmbagent.py: msg: ", msg)
        agent_for_step = agent
        if 'name' in msg:
            # print("\nin cmbagent.py: msg['name']: ", msg['name'])
            agent_for_step = agent_for_step.removesuffix("_context")
            agent_for_step = agent_for_step.removesuffix("_agent")
            if msg['name'] == agent_for_step or msg['name'] == f"{agent_for_step}_nest" or msg['name'] == f"{agent_for_step}_response_formatter":
                # print("\nin cmbagent.py: msg['content']: ", msg['content'])
                this_step_execution_summary = msg['content']
                break

    # print(extract_first_function(this_step_execution_summary))
    src = extract_first_function(this_step_execution_summary)
    return src

In [10]:
func = get_response()

_User (to chat_manager):

### Problem
In cosmology, large-scale cosmological dark-matter halo fields are biased tracers of the underlying Gaussian matter density $\delta_m$. Assume we have a sample $\delta_m$. We simulate a halo number density field by taking $n(\mathbf{x}) = \bar{n}\max(0,1+b\delta_m(\mathbf{x}))$, where bare number density $\bar{n}$ and bare bias $b$ are specified constants. What is the bias of the sampled halo field? Derive an equation to evaluate the bias which depends on the bare bias and the variance in each pixel.

### Requirements
Provide the answer in the form of the \texttt{python} code. Implement the following function.
\begin{python}
#let b_in stand for bare bias
def b_eff(sigma: float, b_in:float) -> float:
    pass
\end{python}

--------------------------------------------------------------------------------

Next speaker: engineer


>>>>>>>> USING AUTO REPLY...
             Model    agent    Cost  Prompt Tokens  Completion Tokens  Total Tokens
gpt-4.1-20

In [11]:


# 1) Syntax-highlighted code pane (recommended)
display(Code(func, language="python"))

def b_eff(sigma: float, b_in: float) -> float:
    r"""
    Compute the effective bias of a halo field sampled as n(x) = nbar * max(0, 1 + b_in * delta_m(x)),
    where delta_m(x) is a Gaussian random field with mean 0 and standard deviation sigma.

    The effective bias is defined as the regression coefficient of n(x) on delta_m(x):
        b_eff = Cov(n, delta_m) / Var(delta_m)
              = E[n * delta_m] / sigma^2

    Args:
        sigma (float): Standard deviation of the Gaussian matter density field per pixel (unitless).
        b_in (float): Bare bias parameter (unitless).

    Returns:
        float: Effective bias of the sampled halo field (unitless).
    """
    if sigma <= 0:
        raise ValueError("sigma must be positive.")
    # Threshold for truncation: 1 + b_in * delta_m >= 0 => delta_m >= -1/b_in
    if b_in == 0:
        return 0.0

    # Truncation point
    x0 = -1.0 / b_in
    # Standardize
    z0 = x0 / sigma

    # Standard normal PDF and CDF at truncation
    phi = norm.pdf(z0)
    Phi = norm.cdf(z0)

    # E[n * delta_m] = nbar * E[max(0, 1 + b_in * delta_m) * delta_m]
    # = nbar * ∫_{x0}^{∞} (1 + b_in * x) * x * P(x) dx
    # = nbar * [∫_{x0}^{∞} x * P(x) dx + b_in * ∫_{x0}^{∞} x^2 * P(x) dx]
    # where P(x) is N(0, sigma^2)
    # The moments of truncated normal:
    # E[x | x > x0] = sigma * phi / (1 - Phi)
    # E[x^2 | x > x0] = sigma^2 * (1 + z0 * phi / (1 - Phi))
    # But we want the *unconditional* expectation, so multiply by (1 - Phi)

    # First moment: E[x * 1_{x > x0}]
    Ex = sigma * phi
    # Second moment: E[x^2 * 1_{x > x0}]
    Ex2 = sigma**2 * (1 - Phi) + x0 * sigma * phi

    # E[n * delta_m] = nbar * [Ex + b_in * Ex2]
    # But nbar cancels in the bias ratio, so we can ignore it

    numerator = Ex + b_in * Ex2
    denominator = sigma**2 * (1 - Phi)  # E[delta_m^2 * 1_{x > x0}] = Var(delta_m) * P(x > x0)

    # The effective bias is the regression coefficient:
    # b_eff = E[n * delta_m] / E[delta_m^2 * 1_{x > x0}]
    # But since n(x) is zero for x < x0, the regression is over the full field
    # The correct formula is:
    # b_eff = numerator / (sigma**2 * (1 - Phi))

    return numerator / denominator

In [17]:
candidate_code = func


In [18]:
evaluate(reference_code,candidate_code)

Output()

{'same_io_dynamic': 0.0, 'content_similarity': 0.11347982945227944}

In [30]:
from scipy.stats import norm                                                    
def b_eff1(sigma: float, b_in:float) -> float:
    alpha = sigma*abs(b_in)
    return b_in*norm.cdf(1/alpha)/(norm.cdf(1/alpha)+alpha*norm.pdf(1/alpha))

In [31]:
def b_eff2(sigma: float, b_in: float) -> float:
    if sigma <= 0:
        raise ValueError("sigma must be positive.")
    # Threshold for truncation: 1 + b_in * delta_m >= 0 => delta_m >= -1/b_in
    if b_in == 0:
        return 0.0

    # Truncation point
    x0 = -1.0 / b_in
    # Standardize
    z0 = x0 / sigma

    # Standard normal PDF and CDF at truncation
    phi = norm.pdf(z0)
    Phi = norm.cdf(z0)

    # E[n * delta_m] = nbar * E[max(0, 1 + b_in * delta_m) * delta_m]
    # = nbar * ∫_{x0}^{∞} (1 + b_in * x) * x * P(x) dx
    # = nbar * [∫_{x0}^{∞} x * P(x) dx + b_in * ∫_{x0}^{∞} x^2 * P(x) dx]
    # where P(x) is N(0, sigma^2)
    # The moments of truncated normal:
    # E[x | x > x0] = sigma * phi / (1 - Phi)
    # E[x^2 | x > x0] = sigma^2 * (1 + z0 * phi / (1 - Phi))
    # But we want the *unconditional* expectation, so multiply by (1 - Phi)

    # First moment: E[x * 1_{x > x0}]
    Ex = sigma * phi
    # Second moment: E[x^2 * 1_{x > x0}]
    Ex2 = sigma**2 * (1 - Phi) + x0 * sigma * phi

    # E[n * delta_m] = nbar * [Ex + b_in * Ex2]
    # But nbar cancels in the bias ratio, so we can ignore it

    numerator = Ex + b_in * Ex2
    denominator = sigma**2 * (1 - Phi)  # E[delta_m^2 * 1_{x > x0}] = Var(delta_m) * P(x > x0)

    # The effective bias is the regression coefficient:
    # b_eff = E[n * delta_m] / E[delta_m^2 * 1_{x > x0}]
    # But since n(x) is zero for x < x0, the regression is over the full field
    # The correct formula is:
    # b_eff = numerator / (sigma**2 * (1 - Phi))

    return numerator / denominator

In [42]:
import math
def b_eff3(sigma: float, b_in: float) -> float:
    r"""Calculates the effective bias of a halo field.

    The halo number density is modeled as n(x) = n_bar * max(0, 1 + b_in * delta_m(x)),
    where delta_m is a Gaussian random field with mean 0 and standard deviation `sigma`.
    The effective bias, b_eff, is derived using the peak-background split formalism.
    This formalism defines the bias as b_eff = d<n_h>/d(delta_L)|_{delta_L=0}, where delta_L 
    is a long-wavelength overdensity mode and the average is over short-scale delta_m.
    The resulting formula is:
    b_eff = b_in * P(1 + b_in * delta_m > 0)

    Let P_positive = P(1 + b_in * delta_m > 0).
    If b_in > 0, P_positive = P(delta_m > -1/b_in) 
                           = P(delta_m/sigma > -1/(b_in*sigma))
                           = 1 - Phi(-1/(b_in*sigma)) 
                           = Phi(1/(b_in*sigma)).
    If b_in < 0, P_positive = P(delta_m < -1/b_in) 
                           = P(delta_m/sigma < -1/(b_in*sigma))
                           = Phi(-1/(b_in*sigma)).
    Both cases can be combined into P_positive = Phi(1 / (fabs(b_in) * sigma)),
    where Phi is the Cumulative Distribution Function (CDF) of the standard normal
    distribution N(0,1), and fabs is the absolute value.
    Thus, the effective bias is b_eff = b_in * Phi(1 / (fabs(b_in) * sigma)).

    Special cases:
    - If b_in is 0, the field is n(x) = n_bar * max(0,1) = n_bar (unbiased), so b_eff = 0.
    - If sigma is 0 (no variance in delta_m, so delta_m = 0 everywhere as mean is 0),
      the peak-background split derivation simplifies.
      Effectively, delta_m (small scale, denoted delta_s in PBS) is 0.
      The bias calculation becomes b_eff = d max(0,1+b_in*delta_L)/d delta_L at delta_L=0.
      This evaluates to b_in * Indicator(1+b_in*delta_L > 0) at delta_L=0,
      which is b_in * Indicator(1 > 0) = b_in * 1 = b_in.

    Args:
        sigma (float): Standard deviation of the matter density field delta_m.
                       It must be non-negative. If variance is var_val, pass sigma=sqrt(var_val).
        b_in (float): The bare bias parameter.

    Returns:
        float: The effective bias.

    Raises:
        TypeError: If sigma or b_in are not numbers.
        ValueError: If sigma is negative.
    """
    if not isinstance(sigma, (int, float)):
        raise TypeError("sigma must be a number (int or float).")
    if not isinstance(b_in, (int, float)):
        raise TypeError("b_in must be a number (int or float).")

    if sigma < 0:
        raise ValueError("sigma (standard deviation) cannot be negative.")

    # Handle special cases first
    if b_in == 0.0:
        return 0.0

    if sigma == 0.0:
        # As derived in the docstring, for sigma = 0, b_eff = b_in.
        # This ensures the function returns a float as per type hint.
        return float(b_in)

    # General case: sigma > 0 and b_in != 0.
    # So, math.fabs(b_in) * sigma > 0, and division by zero is not a risk here.
    abs_b_in_times_sigma = math.fabs(b_in) * sigma
    arg_phi = 1.0 / abs_b_in_times_sigma

    # Phi(arg_phi) is the CDF of the standard normal distribution.
    # scipy.stats.norm.cdf is a common way to compute this.
    # Alternatively, using math.erf:
    # phi_val = 0.5 * (1.0 + math.erf(arg_phi / math.sqrt(2.0)))
    phi_val = norm.cdf(arg_phi)

    return b_in * phi_val

In [55]:
a = 0.55
b = 2
print("b_eff_reference: ", b_eff1(a, b))
print("b_eff_gpt4.1: ", b_eff2(b, b))
print("b_eff_gemini2.5: ", b_eff3(b, b))

b_eff_reference:  1.4763037843329683
b_eff_gpt4.1:  2.0
b_eff_gemini2.5:  1.1974126513658474


In [52]:
Code(reference_code)

from scipy.stats import norm
#let b_in stand for bare bias
def b_eff(sigma: float, b_in:float) -> float:
    alpha = sigma*abs(b_in)
    return b_in*norm.cdf(1/alpha)/(norm.cdf(1/alpha)+alpha*norm.pdf(1/alpha))